In [1]:
import fasttext
model = fasttext.load_model('./util/cc.en.300.bin')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import json
import re

from tqdm import tqdm

C:\Users\Baha Tegar\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
import util.normalization as norm
import util.model as models
import util.utility as util

In [4]:
# Opening JSON file
with open("./clothes_result/map_second_clothes.json") as json_file:
    map_second_keys = json.load(json_file)

with open("./clothes_result/map_category_clothes.json") as json_file:
    map_category = json.load(json_file)

with open("./clothes_result/list_first_cycle_clothes.json") as json_file:
    list_first_cycle = json.load(json_file)

In [5]:
df_base = pd.read_csv("./mcd_result/base_mcd_v2.csv")

print(df_base.info())
df_base.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   reviewer_id       100 non-null    int64 
 1   review            100 non-null    object
 2   summarize         100 non-null    object
 3   aspect            100 non-null    object
 4   token_sentence    100 non-null    object
 5   token_lemma       100 non-null    object
 6   rating            100 non-null    int64 
 7   residu            95 non-null     object
 8   key_score_aspect  98 non-null     object
dtypes: int64(2), object(7)
memory usage: 7.2+ KB
None


,reviewer_id,review,summarize,aspect,token_sentence,token_lemma,rating,residu,key_score_aspect
0,1,Why does it look like someone spit on my food?...,Why does it look like someone spit on my food...,"normal transaction, clear substance, someone s...","Why does it look like someone spit on my food,...","why do it look like someone spit on my food, i...",1,"everyone, clear substance, someone spit, norma...","drink, someone spit, negative; situation, norm..."
1,2,It'd McDonalds. It is what it is as far as the...,The staff at McDonalds are always friendly an...,"atmosphere, staff, food, mcdonalds",The staff at McDonalds are always friendly and...,the staff at mcdonalds be always friendly and ...,4,atmosphere,"place, mcdonalds, positive; food, food, positi..."
2,3,Made a mobile order got to the speaker and che...,Never got the refund in the app. Made a mobil...,"mobile order, line, refund, money","Never got the refund in the app., Made a mobil...","never get the refund in the app ., make a mobi...",1,"refund, money, mobile order","service, mobile order, negative; situation, li..."
3,4,My mc. Crispy chicken sandwich was ÃÂ¯ÃÂ¿ÃÂ...,My mc. Crispy chicken sandwich was customer ...,"chicken sandwich, customer service, customer, ...",Crispy chicken sandwich was customer service ...,crispy chicken sandwich be customer service be...,5,crispy chicken,"service, customer service, positive; food, chi..."
4,5,"I repeat my order 3 times in the drive thru, a...","I repeat my order 3 times in the drive thru, ...","order, large meal, english, close attention, l...","I repeat my order 3 times in the drive thru, i...","i repeat my order 3 time in the drive thru, it...",1,"english, close attention, double filet","situation, large meal/large fry, positive; foo..."


In [6]:
def split_all(x):
    temp = []
    for e in x:
        if len(e) == 0:
            continue
        temp.append(tuple(e.split(", ")))
        
    return temp



df_base = df_base.fillna("")

df_base['aspect'] = df_base['aspect'].apply(lambda x: x.split(", "))
df_base['token_sentence'] = df_base['token_sentence'].apply(lambda x: x.split(", "))
df_base['token_lemma'] = df_base['token_lemma'].apply(lambda x: x.split(", "))
df_base['residu'] = df_base['residu'].apply(lambda x: x.split(", "))

df_base['key_score_aspect'] = df_base['key_score_aspect'].apply(lambda x: x.split("; ")).apply(lambda x: split_all(x))

print(df_base.info())
df_base.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   reviewer_id       100 non-null    int64 
 1   review            100 non-null    object
 2   summarize         100 non-null    object
 3   aspect            100 non-null    object
 4   token_sentence    100 non-null    object
 5   token_lemma       100 non-null    object
 6   rating            100 non-null    int64 
 7   residu            100 non-null    object
 8   key_score_aspect  100 non-null    object
dtypes: int64(2), object(7)
memory usage: 7.2+ KB
None


,reviewer_id,review,summarize,aspect,token_sentence,token_lemma,rating,residu,key_score_aspect
0,1,Why does it look like someone spit on my food?...,Why does it look like someone spit on my food...,"[normal transaction, clear substance, someone ...",[Why does it look like someone spit on my food...,"[why do it look like someone spit on my food, ...",1,"[everyone, clear substance, someone spit, norm...","[(drink, someone spit, negative), (situation, ..."
1,2,It'd McDonalds. It is what it is as far as the...,The staff at McDonalds are always friendly an...,"[atmosphere, staff, food, mcdonalds]",[The staff at McDonalds are always friendly an...,[the staff at mcdonalds be always friendly and...,4,[atmosphere],"[(place, mcdonalds, positive), (food, food, po..."
2,3,Made a mobile order got to the speaker and che...,Never got the refund in the app. Made a mobil...,"[mobile order, line, refund, money]","[Never got the refund in the app., Made a mobi...","[never get the refund in the app ., make a mob...",1,"[refund, money, mobile order]","[(service, mobile order, negative), (situation..."
3,4,My mc. Crispy chicken sandwich was ÃÂ¯ÃÂ¿ÃÂ...,My mc. Crispy chicken sandwich was customer ...,"[chicken sandwich, customer service, customer,...",[Crispy chicken sandwich was customer service...,[crispy chicken sandwich be customer service b...,5,[crispy chicken],"[(service, customer service, positive), (food,..."
4,5,"I repeat my order 3 times in the drive thru, a...","I repeat my order 3 times in the drive thru, ...","[order, large meal, english, close attention, ...","[I repeat my order 3 times in the drive thru, ...","[i repeat my order 3 time in the drive thru, i...",1,"[english, close attention, double filet]","[(situation, large meal/large fry, positive), ..."


In [7]:


df_main = df_base.copy()

df_main['token_sentence'] = df_main['token_sentence'].apply(lambda x: "\n".join("● " + item for item in x))

df_main.head()

,reviewer_id,review,summarize,aspect,token_sentence,token_lemma,rating,residu,key_score_aspect
0,1,Why does it look like someone spit on my food?...,Why does it look like someone spit on my food...,"[normal transaction, clear substance, someone ...",● Why does it look like someone spit on my foo...,"[why do it look like someone spit on my food, ...",1,"[everyone, clear substance, someone spit, norm...","[(drink, someone spit, negative), (situation, ..."
1,2,It'd McDonalds. It is what it is as far as the...,The staff at McDonalds are always friendly an...,"[atmosphere, staff, food, mcdonalds]",● The staff at McDonalds are always friendly a...,[the staff at mcdonalds be always friendly and...,4,[atmosphere],"[(place, mcdonalds, positive), (food, food, po..."
2,3,Made a mobile order got to the speaker and che...,Never got the refund in the app. Made a mobil...,"[mobile order, line, refund, money]",● Never got the refund in the app.\n● Made a m...,"[never get the refund in the app ., make a mob...",1,"[refund, money, mobile order]","[(service, mobile order, negative), (situation..."
3,4,My mc. Crispy chicken sandwich was ÃÂ¯ÃÂ¿ÃÂ...,My mc. Crispy chicken sandwich was customer ...,"[chicken sandwich, customer service, customer,...",● Crispy chicken sandwich was customer servic...,[crispy chicken sandwich be customer service b...,5,[crispy chicken],"[(service, customer service, positive), (food,..."
4,5,"I repeat my order 3 times in the drive thru, a...","I repeat my order 3 times in the drive thru, ...","[order, large meal, english, close attention, ...",● I repeat my order 3 times in the drive thru\...,"[i repeat my order 3 time in the drive thru, i...",1,"[english, close attention, double filet]","[(situation, large meal/large fry, positive), ..."


In [8]:
def custom_text(x):
    temp = {}
    text = ""
    if not x:
        return text
    
    for s in x:
        category, key, sentiment = s

        if category not in temp:
            temp[category] = f"\t- {key} ({sentiment})"
        else:
            temp[category] = temp[category] + "\n" + f"\t- {key} ({sentiment})"



    categories = list(temp.keys())
    for category in categories:
        temp_text = f"● {category}:\n" + temp[category]

        text = text + temp_text + '\n'

    return text

In [9]:
df_main['information'] = df_main['key_score_aspect'].apply(lambda x: custom_text(x))

df_main = df_main.drop('key_score_aspect', axis=1)

In [10]:
columns = ['reviewer_id', 'review', 'summarize', 'token_sentence', 'information', 'rating']
df_main = df_main[columns]

df_main.to_csv('./clothes_result/main_clothes.csv', index=False)